In [23]:
#importing libraries
import torch
import torch.nn as nn

In [24]:
# Dummy dataset: sequence of numbers and their labels (sum of the sequence)
sequences = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
]
labels = [6, 15, 24, 33]  # The sum of each sequence


In [25]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        # output_size should be 1 since we're predicting the sum (a scalar)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        out, _ = self.rnn(x)  # output: (batch_size, seq_len, hidden_size)
        
        # We only need the output from the last time step
        last_hidden = out[:, -1, :]  # shape: (batch_size, hidden_size)
        
        # Predict a scalar value, so output size is 1 (scalar)
        output = self.linear(last_hidden)  # shape: (batch_size, 1)

        return output


In [26]:
#model, loss_fn, optimizer
input_size=1
hidden_size=10
output_size=1

model=RNN(input_size, hidden_size, output_size)

loss_fn=nn.MSELoss()

optimizer=torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

RNN(
  (rnn): RNN(1, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=1, bias=True)
)


In [43]:
#Training loop

num_epochs = 1100
for epoch in range(num_epochs):
    model.train()  
    total_loss = 0  

    for sequence, label in zip(sequences, labels):  
        # Convert sequence to tensor (add batch dimension)
        input_sequence = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
        target = torch.tensor([label], dtype=torch.float32)  # Convert label to tensor

        output= model(input_sequence)

        # Compute loss (mean squared error)
        loss = loss_fn(output, target)  # Squeeze batch dimension after output
        total_loss += loss.item()

        # Backward pass: compute gradients
        loss.backward()

        # Update parameters
        optimizer.step()

        optimizer.zero_grad()

    # Print the average loss after each epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(sequences)}")


Epoch [1/1100], Loss: 13.835240051703295
Epoch [2/1100], Loss: 13.805712706393251
Epoch [3/1100], Loss: 13.776243614164741
Epoch [4/1100], Loss: 13.746839841893461
Epoch [5/1100], Loss: 13.717473663194141
Epoch [6/1100], Loss: 13.688132504009502
Epoch [7/1100], Loss: 13.658801983926423
Epoch [8/1100], Loss: 13.629464874000405
Epoch [9/1100], Loss: 13.600103348632047
Epoch [10/1100], Loss: 13.570671352798854
Epoch [11/1100], Loss: 13.541149641148479
Epoch [12/1100], Loss: 13.5114938525831
Epoch [13/1100], Loss: 13.481656614616952
Epoch [14/1100], Loss: 13.451589006157121
Epoch [15/1100], Loss: 13.421227305438379
Epoch [16/1100], Loss: 13.390488041854269
Epoch [17/1100], Loss: 13.35926101818427
Epoch [18/1100], Loss: 13.327433693493788
Epoch [19/1100], Loss: 13.294843814044725
Epoch [20/1100], Loss: 13.26131225420977
Epoch [21/1100], Loss: 13.22656810026092
Epoch [22/1100], Loss: 13.190324551875676
Epoch [23/1100], Loss: 13.152159510104184
Epoch [24/1100], Loss: 13.111605314283281
Epoch 

In [45]:
# Test the model with some sample sequences
model.eval()

# Test the model on a new sequence
test_sequence = [10,11,12] 
test_input = torch.tensor(test_sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)

with torch.inference_mode():  
    predicted_output = model(test_input)

predicted_sum = predicted_output.item()  # Convert to scalar

print(f"Predicted sum for the sequence {test_sequence}: {predicted_sum}")


Predicted sum for the sequence [10, 11, 12]: 32.97019577026367
